## Ejercicios de análisis exploratorio de datos con Python Pandas

- [Ejercicio 1](#ejercicio-1): 2017 1er cuatrimestre (primera oportunidad)
- [Ejercicio 2](#ejercicio-2): 2017 2do cuatrimestre (primera oportunidad)
- [Ejercicio 3](#ejercicio-3): 2017 2do cuatrimestre (segunda oportunidad)
- [Ejercicio 4](#ejercicio-4): 2018 1er cuatrimestre (primera oportunidad)
- [Ejercicio 5](#ejercicio-5): 2018 1er cuatrimestre (primera oportunidad)
- [Ejercicio 6](#ejercicio-6): 2018 1er cuatrimestre (segunda oportunidad)

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import pyspark
import math

### Ejercicio 1

** 2017 1er cuatrimestre (primera oportunidad)**

Un sitio de Ebooks tiene información sobre los reviews que los usuarios hacen de sus libros en un DataFrame con formato (user_id, book_id, rating, timestamp). Por otro lado tenemos información en otro DataFrame que bajamos de GoodReads: (book_id, book_name, avg_rating). Podemos suponer que los Ids de los libros son compatibles. Se pide usar Python Pandas para:

  a) obtener un DataFrame que indique el TOP5 de Ebooks en el sitio de Ebooks.
(Para este punto se puede ignorar el segundo DataFrame) (7,5 ptos);

  b) obtener un DataFrame que indique qué libros tienen una diferencia de rating
promedio mayor al 20% entre el sitio de Ebooks y GoodReads (7,5 ptos).

#### Criterio de corrección

Para el punto a) Si no tiene en cuenta un mínimo de ratings por libro antes de considerar el promedio -3. Si
agrupa mal o calcula mal el promedio -5, si la lógica está mal vale cero. Errores mínimos de sintaxis no serán tenidos
en cuenta, pero si utilizan funciones no existentes se descuentan puntos en función de que hagan. Para el punto b)
Al igual que antes, si calculan el promedio sin evaluar la cantidad de ratings para un book_id dado -3, si hace mal el
join/merge -5, si hacen mal la validación de la condición -3.


#### Lectura de los datos

In [2]:
ebooks = pd.read_csv('../data/ebooks/ebooks.csv', encoding = 'utf-8')
ebooks.head()

,user_id,book_id,rating,timestamp
0,4,2,9,2017-05-04
1,2,1,3,2017-05-03
2,2,2,5,2017-09-21
3,3,1,6,2018-01-15
4,5,3,2,2016-12-24


In [3]:
goodreads = pd.read_csv('../data/ebooks/goodreads.csv', encoding = 'utf-8')
goodreads.head()

,book_id,book_name,avg_rating
0,1,20000 leguas de viaje submarino,8.4
1,2,Estudio en escarlata,2.3
2,3,La isla misteriosa,9.2
3,4,La isla del tesoro,7.6
4,5,Las aventuras de Tom Sawyer,6.7


#### a) Top 5 del primer data frame

In [4]:
# Me quedo con las columnas que me interesan
dfa = ebooks.loc[:,['book_id', 'rating']]

# Filtro los que tienen un solo review
dfa = dfa.groupby('book_id').filter(lambda x: len(x) > 1)

# Agrupo por book_id, calculo el promedio, ordeno
# descendientemente y tomo los primeros cinco
dfa.groupby('book_id').mean()\
    .sort_values('rating', ascending = False).head(5)

,rating
book_id,
4,7.0
2,6.5
5,6.5
7,5.5
1,5.0


##### Otra forma

In [5]:
# Me quedo con las columnas que me interesan
dfa = ebooks.loc[:,['book_id', 'rating']]

# Agrupo por libro, cuento los votos y obtengo una
# serie con la cantidad de votos por libro
num_reviews = dfa.groupby('book_id').count()['rating']

# Agrupo por libro y calculo el promedio del puntaje
dfa = dfa.groupby('book_id').mean()

# Agrego la columna de cantidad de reviews
dfa['reviews'] = num_reviews

# Descarto los libros que tengan un solo review
dfa = dfa.loc[dfa['reviews'] > 1,:]

# Ordeno descendentemente y muestro los primeros 5
dfa.sort_values(by = 'rating', ascending = False).head(5)

,rating,reviews
book_id,,
4,7.0,3
2,6.5,6
5,6.5,2
7,5.5,2
1,5.0,5


#### b) Libros con una diferencia de rating promedio mayor al 20% entre ambos sitios

In [6]:
# Hago un inner join entre ambos data frames sobre el book_id
merged = pd.merge(dfa.reset_index(), goodreads, on = 'book_id', how = 'inner')
merged

,book_id,rating,reviews,book_name,avg_rating
0,1,5.000000,5,20000 leguas de viaje submarino,8.4
1,2,6.500000,6,Estudio en escarlata,2.3
2,3,2.666667,6,La isla misteriosa,9.2
3,4,7.000000,3,La isla del tesoro,7.6
4,5,6.500000,2,Las aventuras de Tom Sawyer,6.7
5,7,5.500000,2,De la Tierra a la Luna,4.3


In [7]:
# Me quedo solo con aquellos resultados que tengan una diferencia
# mayor a 20% entre ambas columnas de rating
merged.loc[(merged['avg_rating'] < merged['rating'] * 0.8)\
          | (merged['avg_rating'] > merged['rating'] * 1.2),:]

,book_id,rating,reviews,book_name,avg_rating
0,1,5.000000,5,20000 leguas de viaje submarino,8.4
1,2,6.500000,6,Estudio en escarlata,2.3
2,3,2.666667,6,La isla misteriosa,9.2
5,7,5.500000,2,De la Tierra a la Luna,4.3


### Ejercicio 2

** 2017 2do cuatrimestre (primera oportunidad) **

Tenemos un dataframe con la información de distintas playlists armadas por usuarios con el formato (playlist, song_id, description). A su vez, contamos con un dataframe de canciones
que contiene (song_id, singer, year, length, genres).
Se pide generar un programa en Pandas que indique para cada playlist cuál es el cantante predominante (con mas canciones incluidas dentro de esa lista). (15 pts) 

#### Criterio de corrección

Si en algún lado hacen un groupByKey se hace un descuento de 10 puntos. En ningún caso son necesarios los datos de cada registro.
Si filtran después de hacer el resto de las operaciones, descuento de 2 puntos (es mucho más eficiente filtrar antes y solo trabajar con un
conjunto acotado, podríamos tener todos los patentamientos de la historia). 

#### Lectura de los datos

In [8]:
playlists = pd.read_csv('../data/playlists/playlists.csv', encoding='utf-8')
playlists.head()

,playlist,song_id,description
0,1,3,werwe
1,1,5,ijias
2,4,4,oiurewq
3,1,4,fdsa
4,2,4,rewqew


In [9]:
songs = pd.read_csv('../data/playlists/songs.csv', encoding='utf-8')
songs.head()

,song_id,singer,year,length,genres
0,1,Michael Jackson,1985,8,Pop
1,2,Elton John,1984,2,Pop
2,3,Ronnie Dio,1987,3,Metal
3,4,Elton John,1982,4,Pop
4,5,Elton John,1985,3,Pop


#### Resolución agrupando y buscando el máximo

In [10]:
# Me deshago primero de las columnas que no me interesan
playlists = playlists.loc[:,['playlist', 'song_id']]
songs = songs.loc[:, ['song_id', 'singer']]

# Hago un inner join sobre song_id
merged = pd.merge(playlists, songs, how = 'inner', on = 'song_id')
merged.head()

,playlist,song_id,singer
0,1,3,Ronnie Dio
1,2,3,Ronnie Dio
2,3,3,Ronnie Dio
3,1,5,Elton John
4,2,5,Elton John


In [11]:
# Hago un groupby por playlist y singer y luego me quedo con el máximo de cada uno
grouped = merged.groupby(['playlist', 'singer'])\
    .size().to_frame('cant').reset_index()
grouped

,playlist,singer,cant
0,1,Elton John,2
1,1,Ronnie Dio,1
2,2,Elton John,3
3,2,Ronnie Dio,1
4,3,Elton John,1
5,3,Ronnie Dio,1
6,4,Elton John,2
7,5,Michael Jackson,2


In [12]:
# Me quedo con el máximo por cada playlist
grouped.groupby('playlist')\
    .apply(lambda x: x[x['cant'] == x['cant'].max()])

playlist           singer  cant
playlist                                   
1        0         1       Elton John     2
2        2         2       Elton John     3
3        4         3       Elton John     1
         5         3       Ronnie Dio     1
4        6         4       Elton John     2
5        7         5  Michael Jackson     2

#### Resolución agrupando, ordenando y eliminando duplicados

In [13]:
# Me deshago primero de las columnas que no me interesan
playlists = playlists.loc[:,['playlist', 'song_id']]
songs = songs.loc[:, ['song_id', 'singer']]

# Hago un inner join sobre song_id
merged = pd.merge(playlists, songs, how = 'inner', on = 'song_id')

# Agrupo por playlist y singer, los cuento, ordeno
# descendientemente y elimino los duplicados
df = merged.groupby(['playlist','singer']).size()\
    .to_frame('cantidad').reset_index()\
    .sort_values('cantidad', ascending = False)\
    .drop_duplicates('playlist')

df.set_index('playlist')

,singer,cantidad
playlist,,
2,Elton John,3
1,Elton John,2
4,Elton John,2
5,Michael Jackson,2
3,Elton John,1


#### Resolución con Spark

Es importante no usar `collect()` cada vez. Aquí lo utilizamos para poder visualizar el resultado de cada paso y porque sabemos que estamos trabajando con un conjunto de datos pequeño, pero en la solución final no debería aparecer.

In [60]:
# Leo el CSV de playlists
playlistsRDD = sc.textFile('../data/playlists/playlists.csv')

# Remuevo la línea de encabezado
header = playlistsRDD.first()
playlistsRDD = playlistsRDD.filter(lambda line: line != header)

# Separo por comas
playlistsRDD = playlistsRDD.map(lambda a: a.split(','))
playlistsRDD.collect()

[['1', '3', 'werwe'],
 ['1', '5', 'ijias'],
 ['4', '4', 'oiurewq'],
 ['1', '4', 'fdsa'],
 ['2', '4', 'rewqew'],
 ['2', '3', 'ewrqwq'],
 ['2', '2', 'ewqor'],
 ['2', '5', 'rewqew'],
 ['3', '3', 'oiure'],
 ['3', '2', 'rewh'],
 ['4', '2', 'wreqoiu'],
 ['5', '6', 'rewew'],
 ['5', '1', 'miguel']]

In [61]:
# (playlist, song_id, description) (0,1,2)
# (song_id, singer, year, length, genres) (0,1,2,3,4)

# Me quedo con las columnas 
# song_id y playlist poniendo song_id primera
playlistsRDD = playlistsRDD.map(lambda x: (x[1],x[0]))
playlistsRDD.collect()

[('3', '1'),
 ('5', '1'),
 ('4', '4'),
 ('4', '1'),
 ('4', '2'),
 ('3', '2'),
 ('2', '2'),
 ('5', '2'),
 ('3', '3'),
 ('2', '3'),
 ('2', '4'),
 ('6', '5'),
 ('1', '5')]

In [62]:
# Leo el CSV de canciones
songsRDD = sc.textFile('../data/playlists/songs.csv')

# Remuevo la línea de encabezado
header = songsRDD.first()
songsRDD = songsRDD.filter(lambda line: line != header)

# Separo por comas
songsRDD = songsRDD.map(lambda a: a.split(','))
songsRDD.collect()

[['1', 'Michael Jackson', '1985', '8', 'Pop'],
 ['2', 'Elton John', '1984', '2', 'Pop'],
 ['3', 'Ronnie Dio', '1987', '3', 'Metal'],
 ['4', 'Elton John', '1982', '4', 'Pop'],
 ['5', 'Elton John', '1985', '3', 'Pop'],
 ['6', 'Michael Jackson', '1990', '5', 'Pop']]

In [63]:
# Me quedo con las primeras dos columnas del segundo RDD
songsRDD = songsRDD.map(lambda x: (x[0],x[1]))
songsRDD.collect()

[('1', 'Michael Jackson'),
 ('2', 'Elton John'),
 ('3', 'Ronnie Dio'),
 ('4', 'Elton John'),
 ('5', 'Elton John'),
 ('6', 'Michael Jackson')]

In [64]:
# Hago un inner join sobre el song_id
merged = songsRDD.join(playlistsRDD)
merged.collect()

[('4', ('Elton John', '4')),
 ('4', ('Elton John', '1')),
 ('4', ('Elton John', '2')),
 ('3', ('Ronnie Dio', '1')),
 ('3', ('Ronnie Dio', '2')),
 ('3', ('Ronnie Dio', '3')),
 ('6', ('Michael Jackson', '5')),
 ('1', ('Michael Jackson', '5')),
 ('2', ('Elton John', '2')),
 ('2', ('Elton John', '3')),
 ('2', ('Elton John', '4')),
 ('5', ('Elton John', '1')),
 ('5', ('Elton John', '2'))]

In [65]:
merged = merged.map(lambda x: ((x[1][1],x[1][0]),1))
merged.collect()

[(('4', 'Elton John'), 1),
 (('1', 'Elton John'), 1),
 (('2', 'Elton John'), 1),
 (('1', 'Ronnie Dio'), 1),
 (('2', 'Ronnie Dio'), 1),
 (('3', 'Ronnie Dio'), 1),
 (('5', 'Michael Jackson'), 1),
 (('5', 'Michael Jackson'), 1),
 (('2', 'Elton John'), 1),
 (('3', 'Elton John'), 1),
 (('4', 'Elton John'), 1),
 (('1', 'Elton John'), 1),
 (('2', 'Elton John'), 1)]

In [66]:
merged = merged.reduceByKey(lambda a,b: a + b)
merged.collect()

[(('1', 'Elton John'), 2),
 (('2', 'Ronnie Dio'), 1),
 (('2', 'Elton John'), 3),
 (('1', 'Ronnie Dio'), 1),
 (('5', 'Michael Jackson'), 2),
 (('4', 'Elton John'), 2),
 (('3', 'Ronnie Dio'), 1),
 (('3', 'Elton John'), 1)]

In [67]:
merged = merged.map(lambda x: (x[0][0],(x[0][1],x[1])))
merged.collect()

[('1', ('Elton John', 2)),
 ('2', ('Ronnie Dio', 1)),
 ('2', ('Elton John', 3)),
 ('1', ('Ronnie Dio', 1)),
 ('5', ('Michael Jackson', 2)),
 ('4', ('Elton John', 2)),
 ('3', ('Ronnie Dio', 1)),
 ('3', ('Elton John', 1))]

In [74]:
aux = merged.map(lambda x: (x[0],x[1][1]))
aux.collect()

[('1', 2),
 ('2', 1),
 ('2', 3),
 ('1', 1),
 ('5', 2),
 ('4', 2),
 ('3', 1),
 ('3', 1)]

In [75]:
aux = aux.reduceByKey(max)
aux.collect()

[('4', 2), ('3', 1), ('1', 2), ('2', 3), ('5', 2)]

In [77]:
aux = aux.map(lambda x: ((x[0],x[1]),1))
aux.collect()

[(('4', 2), 1), (('3', 1), 1), (('1', 2), 1), (('2', 3), 1), (('5', 2), 1)]

In [78]:
merged = merged.map(lambda x: ((x[0],x[1][1]), x[1][0]))
merged.collect()

[(('1', 2), 'Elton John'),
 (('2', 1), 'Ronnie Dio'),
 (('2', 3), 'Elton John'),
 (('1', 1), 'Ronnie Dio'),
 (('5', 2), 'Michael Jackson'),
 (('4', 2), 'Elton John'),
 (('3', 1), 'Ronnie Dio'),
 (('3', 1), 'Elton John')]

In [81]:
fin = aux.join(merged)
fin.collect()

[(('3', 1), (1, 'Ronnie Dio')),
 (('3', 1), (1, 'Elton John')),
 (('2', 3), (1, 'Elton John')),
 (('5', 2), (1, 'Michael Jackson')),
 (('4', 2), (1, 'Elton John')),
 (('1', 2), (1, 'Elton John'))]

In [82]:
fin = fin.map(lambda x: (x[0][0], x[1][1]))
fin.collect()

[('3', 'Ronnie Dio'),
 ('3', 'Elton John'),
 ('2', 'Elton John'),
 ('5', 'Michael Jackson'),
 ('4', 'Elton John'),
 ('1', 'Elton John')]

### Ejercicio 3

** 2017 2do cuatrimestre (segunda oportunidad) **

La Agencia Nacional de Estadísticas de Buenos Aires recolecta información de nacimientos cuando los padres registran a sus hijos en el registro civil a partir de una encuesta. Esa información se encuentra disponible para su análisis en un csv con el siguiente formato
(dia_nacimiento, mes_nacimiento, anio_nacimiento, peso_al_nacer, longitud_al_nacer, id_hospital, tipo_parto), donde el tipo de parto 1 es natural y 2 es cesárea.
Por otro lado la agencia cuenta con información histórica de los hospitales en otro csv con siguiente formato (id_hospital, dirección, promedio_nacimientos_mensual).
Se pide usar Pandas para:

  a) Calcular la cantidad de nacimientos para cada uno de los hospitales para el mes de Octubre de 2017 e indicar aquellos hospitales que superan el promedio de nacimientos mensuales.
  
  b) Comparando el mes de Octubre de 2017 indicar programáticamente si se incremento el % de cesáreas con respecto a ese mes del año 2016.
  
#### Criterio de corrección

El enunciado dice que se cuenta con dos csv, por lo que hay que comenzar leyéndolos. a) Por un lado es necesario filtrar del Data frame
aquellos datos que corresponden al mes de Octubre de 2017. Para obtener la cantidad de nacimientos, por hospital se puede hacer de muchas
formas, una es agrupar y usar size, otra opción seria generar una columna con 1 por cada nacimiento y realizar una agrupación por hospital
realizando un aggregate(sum) de esa columna para tener el total por hospital.
Luego es necesario realizar el un inner join con la información histórica de hospitales y filtrar aquellas que no cuyo total sea menor o igual al
promedio histórico de nacimientos.
b) Se debe trabajar respectivamente filtrando datos del data frame para los meses de octubre de los años 2016 y 2017. Es necesario obtener el
total de nacimientos para cada uno de los meses y el total de cesáreas de ambos casos para poder calcular el porcentaje para cada año.
Es importante que se determine sí se incrementó o no el porcentaje. Si no se realiza descuento de 4 puntos.

#### Lectura de datos

In [14]:
nacimientos = pd.read_csv('../data/nacimientos/nacimientos.csv')
nacimientos.head()

,dia_nacimiento,mes_nacimiento,anio_nacimiento,peso_al_nacer,longitud_al_nacer,id_hospital,tipo_parto
0,1,10,2016,3,0.5,4,1
1,2,10,2017,3,0.5,4,2
2,3,9,2005,3,0.5,4,1
3,4,8,2017,3,0.5,4,1
4,5,10,2016,3,0.5,4,1


In [15]:
historico = pd.read_csv('../data/nacimientos/historico_mensual.csv')
historico.head()

,id_hospital,dirección,promedio_nacimientos_mensual
0,4,Rivadavia 1234,5
1,1,9 de Julio 1000,4
2,3,Evergreen 123,8
3,2,General Paz 984,10


#### a) Nacimientos en octubre de 2017 que superan el histórico mensual

In [16]:
# Me quedo con los nacimientos en octubre de 2017 
# y con las columnas id_hospital y tipo_parto
nacs = nacimientos.loc[(nacimientos.anio_nacimiento == 2017) & \
                        (nacimientos.mes_nacimiento == 10),\
                        ['id_hospital', 'tipo_parto']]
nacs.head()

,id_hospital,tipo_parto
1,4,2
6,4,1
8,4,1
15,4,1
17,4,1


In [17]:
# Agrupo por hospitales y calculo la cantidad
nacs = nacs.groupby('id_hospital').size().to_frame('cant').reset_index()
nacs

,id_hospital,cant
0,1,2
1,3,1
2,4,7


In [18]:
# Hago un inner join con el dataframe histórico
merged = pd.merge(nacs, historico, on = 'id_hospital', how = 'inner')

# Me quedo con los hospitales que superen el histórico mensual
merged.loc[merged.cant > merged.promedio_nacimientos_mensual,:]

,id_hospital,cant,dirección,promedio_nacimientos_mensual
2,4,7,Rivadavia 1234,5


#### b) Incremento de cesáreas entre octubre de 2016 y 2017

In [19]:
nacs10 = nacimientos.loc[((nacimientos.anio_nacimiento == 2016) |
                (nacimientos.anio_nacimiento == 2017)) & \
                (nacimientos.mes_nacimiento == 10), \
                ['anio_nacimiento','tipo_parto']]
nacs10.head()

,anio_nacimiento,tipo_parto
0,2016,1
1,2017,2
4,2016,1
6,2017,1
7,2016,1


In [21]:
nacs16 = nacimientos.loc[(nacimientos.anio_nacimiento == 2016) & \
                  (nacimientos.mes_nacimiento == 10), ['tipo_parto']]
nacs16.head()

,tipo_parto
0,1
4,1
7,1
10,2
13,1


In [22]:
nacs16 = nacs16.groupby('tipo_parto').size().to_frame('cant')
nacs16

,cant
tipo_parto,
1,5
2,2


In [23]:
nacs16['perc'] = nacs16.apply(lambda x: x.cant / nacs16.cant.agg(sum), axis = 1)
nacs16

,cant,perc
tipo_parto,,
1,5,0.714286
2,2,0.285714


In [25]:
nacs17 = nacimientos.loc[(nacimientos.anio_nacimiento == 2017) & \
                  (nacimientos.mes_nacimiento == 10), ['tipo_parto']]
nacs17 = nacs17.groupby('tipo_parto').size().to_frame('cant')
nacs17['perc'] = nacs17.apply(lambda x: x.cant / nacs17.cant.agg(sum), axis = 1)
nacs17

,cant,perc
tipo_parto,,
1,5,0.5
2,5,0.5


In [26]:
merged = pd.merge(nacs16.reset_index(), nacs17.reset_index(),\
                  on = 'tipo_parto', how = 'inner')
merged

,tipo_parto,cant_x,perc_x,cant_y,perc_y
0,1,5,0.714286,5,0.5
1,2,2,0.285714,5,0.5


### Ejercicio 4

** 2018 1er cuatrimestre (primera oportunidad) **

El GCPD (Gotham City Police Department) recolecta la información de casos policiales que acontecen en 
Ciudad Gótica. Esta información se encuentra guardada en un dataframe con el siguiente formato: (fecha, id_caso, descripcion, estado_caso, categoria, latitud, longitud).
Los  posibles estados  que puede tener  un caso son 1: caso abierto, 2: caso  resuelto, 3: cerrado sin resolución.  Las fechas se encuentran en el formato YYYY-MM-DD. 
Por otro lado el comisionado Gordon guarda un registro detallado sobre en cuáles casos fue activada la
 batiseñal para pedir ayuda del vigilante, Batman. Esta información se encuentra en un Dataframe con el siguiente formato (id_caso, respuesta), siendo campo respuesta si la señal tuvo una respuesta positiva (1) o negativa (0) de parte de él.
El  sector  encargado  de  las  estadísticas  oficiales  del  GCPD  quiere  con  esta  información  analizar  las  siguientes situaciones: 

  a) Tasa de resolución de casos de la fuerza policial por categoría de caso (considerando aquellos casos en los que no participó Batman). 
  
  b) Tasa de resolución de casos con la ayuda de Batman (considerando que aquellos casos en los que fue llamado con la batiseñal, participó en la resolución). 
  
  c) Indicar el mes del año pasado en el que Batman tuvo mayor participación en la investigación de casos. 

 
#### Criterio de corrección

Una opción para la resolución es  generar una nueva columna en el Data Frame que podamos usar para 
saber en qué casos estuvo involucrado Batman. Para ello se puede realizar un join filtrando aquellos casos que tuvieron respuesta a la batiseñal (respuesta en 1), más allá de que hay distintas formas de hacerlo. Se descuentan puntos si la solución no es eficiente, o si se intentan realizar a mano operaciones que ya están resueltas por funciones de pandas.

#### Lectura de datos

In [8]:
casos = pd.read_csv('../data/gcpd/casos.csv', encoding = 'utf-8', parse_dates=['fecha'])
casos.head()

,fecha,id_caso,descripcion,estado_caso,categoria,latitud,longitud
0,2016-09-06,1,Robo en callejón sin salida,1,robo,-34.6,-58.38
1,2016-09-07,2,Asesinato en callejón sin salida,2,asesinato,-34.6,-58.38
2,2016-10-08,3,Asalto a banco,3,robo,-34.6,-58.38
3,2016-10-16,4,Femicidio,1,femicidio,-34.6,-58.38
4,2016-11-06,5,Asalto con toma de rehenes,2,robo,-34.6,-58.38


In [10]:
batisenial = pd.read_csv('../data/gcpd/batisenial.csv')
batisenial.head()

,id_caso,respuesta
0,21,1
1,22,1
2,23,1
3,24,0
4,25,1


In [11]:
# Elimino las columnas que no me interesan
casos = casos.loc[:,['id_caso','fecha','estado_caso','categoria']]

# Hago un left join agregando una columna con la participación de Batman
casos = pd.merge(casos, batisenial, on='id_caso', how='left')

# Renombro la nueva columna
casos.rename(columns={'respuesta': 'con_batman'}, inplace=True)

casos.head()

,id_caso,fecha,estado_caso,categoria,con_batman
0,1,2016-09-06,1,robo,NaN
1,2,2016-09-07,2,asesinato,NaN
2,3,2016-10-08,3,robo,NaN
3,4,2016-10-16,1,femicidio,NaN
4,5,2016-11-06,2,robo,NaN


a) Tasa de resolución de casos de la fuerza policial por categoría de caso

In [12]:
# Creo un nuevo data frame con los casos en los que no participó Batman
sin_batman = casos.loc[casos.con_batman != 1,:]
del sin_batman['con_batman']
sin_batman.head()

,id_caso,fecha,estado_caso,categoria
0,1,2016-09-06,1,robo
1,2,2016-09-07,2,asesinato
2,3,2016-10-08,3,robo
3,4,2016-10-16,1,femicidio
4,5,2016-11-06,2,robo


In [13]:
# Agrupo por categoría y calculo la tasa de cada una
sin_batman.groupby('categoria')\
    .apply(lambda x: len(x[x['estado_caso'] == 2]) / len(x))\
    .to_frame('tasa')

,tasa
categoria,
asesinato,0.500000
femicidio,0.500000
hurto,0.333333
incendio intencional,0.500000
narcotráfico,0.500000
robo,0.333333
robo de vehiculos,0.000000
tránsito,0.000000
venta de armas,0.000000


b) Tasa de resolución de casos con la ayuda de Batman

In [14]:
con_batman = casos.loc[casos.con_batman == 1,:]
con_batman = con_batman.drop(columns=['categoria','con_batman'])
cant_casos_con_batman = len(con_batman)
cant_casos_resueltos_con_batman = len(con_batman.loc[con_batman.estado_caso == 2,:])
tasa_resolucion_batman =  cant_casos_resueltos_con_batman / cant_casos_con_batman
print(tasa_resolucion_batman)

0.6470588235294118


c) Mes del año pasado en el que Batman tuvo mayor participación en la investigación de casos. 

In [15]:
# Filtro los casos del año pasado en los que ayudó Batman
con_batman = con_batman.loc[con_batman.fecha.dt.year == 2017,:]

# Creo una nueva columna para el mes
con_batman['mes'] = con_batman['fecha'].dt.month

# Elimino columnas que ya no me interesan
con_batman = con_batman.drop(columns=['fecha','estado_caso'])

# Calculo las cantidades por cada mes, ordeno descendentemente y muestro el primero
con_batman.groupby('mes').size().to_frame('cant')\
    .sort_values('cant', ascending=False).head(1)

,cant
mes,
7,2


### Ejercicio 5

** 2018 1er cuatrimestre (primera oportunidad) **

El Fiscal de Distrito Harvey Dent no esta convencido de que la irrupción de Batman en ciudad Gótica le haya significado a la población y al departamento de policía una mejora en la lucha contra el crimen organizado (categoría número 10 en el dataframe de casos). 
Es tu misión ayudar al Comisionado Gordon planteando una visualización para demostrar  a  lo  largo  del  tiempo  como  fue  evolucionando  la  lucha  contra  el crimen a partir de la participación de Batman, y el valor que le brinda al GCPD su ayuda
 
#### Criterio de corrección

Existen distintos enfoques que se pueden encarar, pero a partir del enunciado se puede ver claramente que el mismo puede considerarse una visualización que nos permita mostrar una serie temporal en la que se pueden mostrar: 
  
- Variación de la tasa de resolución de crímenes en una cierta categoría con la participación de Batman. 
- Relación entre crímenes sin resolver y crímenes resueltos a lo largo del tiempo desde la aparición de Batman. (que podrían por ejemplo mostrarse para una categoría específica que se quiera indicar como la de crimen organizado).

Esto puede lograrse con line plots de y axes, bar plots o stacked bar plots dependiendo de que se quiera mostrar. 

La propuesta debe comunicar efectivamente lo que se desee mostrar. Descuento de puntaje si la visualización no tiene título, ni ejes rotulados. 

### Ejercicio 6

** 2018 1er cuatrimestre (segunda oportunidad) **

El GCPD (Gotham City Police Department) recolecta la información de casos policiales que acontecen en 
Ciudad Gótica. Esta información se encuentra guardada en un dataframe con el siguiente formato: (fecha, id_caso, descripcion, estado_caso, categoria, latitud, longitud).
Los  posibles estados  que puede tener  un caso son 1: caso abierto, 2: caso  resuelto, 3: cerrado sin resolución.  Las fechas se encuentran en el formato YYYY-MM-DD. 
Por otro lado el comisionado Gordon guarda un registro detallado sobre en cuáles casos fue activada la
 batiseñal para pedir ayuda del vigilante, Batman. Esta información se encuentra en un Dataframe con el siguiente formato (id_caso, respuesta), siendo campo respuesta si la señal tuvo una respuesta positiva (1) o negativa (0) de parte de él.
El  sector  encargado  de  las  estadísticas  oficiales  del  GCPD  quiere  con  esta  información  analizar  las  siguientes situaciones: 

  a) Las categorías que hayan incrementado su tasa de resolución en al menos un 10% en el último trimestre con respecto al trimestre anterior.

  b) Tasa de participación de Batman por categoría para los delitos contra la propiedad (que enmarcan las categorías incendio intencional, robo, hurto y robo de vehículos)

 
#### Criterio de corrección

a) Tienen que comparar los totales por trimestre, por lo que lo más simple sería generar una nueva columna con el número de trimestre para los últimos 6 meses (o sea, se quedan con este trimestre y el anterior). Hay varias alternativas de resolución. Una opción es agrupar por categoría y número de trimestre y calcular la tasa de resolución por trimestre para cada categoría. Luego necesitarían hacer un unstack() para quedarse con categoría, tasa_trimestre_actual y tasa_trimestre_anterior como columnas. Una vez con esto es solo cuestión de calcular si el actual supera en un 10% al anterior. Otra opción es usar un transform para calcular la diferencia. También pueden generar un dataframe por trimestre para luego mergearlos pero claramente es peor y tendría un descuento mínimo de 5 puntos. Si no filtran los datos al principio hay un descuento de tres puntos. Operaciones innecesarias tienen un descuento de 3 a 5 puntos.

b) Acá podrían filtrar los registros que contienen alguna de las 4 categorías listadas (incendio intencional, robo, hurto y robo de vehículos) con str.contain(). Si no se filtran los datos antes de comenzar hay un descuento de 3 puntos. Luego deben hacer un join con la participación de Batman para armar una columna 1/0 según haya o no participado Batman, agrupar por categoría y calcular el promedio de la participación.

#### Lectura de datos

In [11]:
casos = pd.read_csv('../data/gcpd/casos.csv', encoding = 'utf-8', parse_dates=['fecha'])
batisenial = pd.read_csv('../data/gcpd/batisenial.csv')

a) Categorías que hayan incrementado su tasa de resolución en al menos un 10% en el último trimestre respecto al anterior.

In [12]:
hoy = pd.to_datetime('today')

# Me quedo con los registros de los últimos seis meses y con las columnas que me interesan
casos_trim = casos.loc[hoy - casos.fecha <= dt.timedelta(days=180),\
                      ['fecha','id_caso','estado_caso','categoria']]

# Creo una nueva columna con el nombre del trimestre
casos_trim['trimestre'] = casos_trim['fecha']\
    .apply(lambda x: 'actual' if ((hoy - x) <= dt.timedelta(days=90)) else 'anterior')

del casos_trim['fecha']

casos_trim

,id_caso,estado_caso,categoria,trimestre
29,30,2,venta de armas,anterior
30,31,2,hurto,anterior
31,32,2,asesinato,anterior
32,33,3,robo,anterior
33,34,1,femicidio,anterior
34,35,2,femicidio,anterior
35,36,2,robo,anterior
36,37,2,robo de vehiculos,anterior
37,38,1,incendio intencional,actual
38,39,2,hurto,actual


In [13]:
# Agrupo por categoría y trimestre, calculo la tasa de resolución para cada agrupamiento
# y pongo el resultado de cada trimestre en columnas separadas mediante unstack
casos_trim = casos_trim.groupby(['categoria','trimestre'])\
    .apply(lambda x: len(x[x['estado_caso'] == 2]) / len(x))\
    .unstack().reset_index()

casos_trim

trimestre,categoria,actual,anterior
0,asesinato,NaN,1.0
1,femicidio,1.0,0.5
2,hurto,1.0,1.0
3,incendio intencional,0.0,NaN
4,narcotráfico,1.0,NaN
5,robo,NaN,0.5
6,robo de vehiculos,NaN,1.0
7,venta de armas,1.0,1.0


In [98]:
# Selecciono las categorías que tengan un incremente superior al 10% en
# la tasa de resolución del trimestre actual respecto al anterior
casos_trim.loc[(casos_trim['actual'] / casos_trim['anterior']) >= 1.1,:]

trimestre,categoria,actual,anterior
1,femicidio,1.0,0.5


b) Tasa de participación de Batman por categoría para los delitos contra la propiedad

In [25]:
# Filtro los casos de las categorías en cuestión y elmino columnas que no me interesan
casos_contra_prop = casos.loc[((casos.categoria.str.contains('robo')) |
                               (casos.categoria.str.contains('incendio intencional')) |\
                               (casos.categoria.str.contains('hurto'))),\
                              ['id_caso','categoria']]

# Hago un left join agregando una columna con la participación de Batman
casos_contra_prop = pd.merge(casos_contra_prop, batisenial, on='id_caso', how='left')

# Agrupo por categoría y calculo la tasa de cada una
casos_contra_prop.groupby('categoria')\
    .apply(lambda x: len(x[x['respuesta'] == 1]) / len(x))\
    .to_frame('tasa')

,tasa
categoria,
hurto,0.500000
incendio intencional,0.333333
robo,0.333333
robo de vehiculos,0.500000


In [4]:
# Leo el CSV de casos
casosRDD = sc.textFile('../data/gcpd/casos.csv')
casosRDD.collect()

# Remuevo la línea de encabezado
header = casosRDD.first()
casosRDD = casosRDD.filter(lambda line: line != header)

# Separo por comas
casosRDD = casosRDD.map(lambda a: a.split(','))

header

'fecha,id_caso,descripcion,estado_caso,categoria,latitud,longitud'

In [5]:
# Leo el CSV de batiseñal
batisenialRDD = sc.textFile('../data/gcpd/batisenial.csv')
batisenialRDD.collect()

# Remuevo la línea de encabezado
header = batisenialRDD.first()
batisenialRDD = batisenialRDD.filter(lambda line: line != header)

# Separo por comas
batisenialRDD = batisenialRDD.map(lambda a: a.split(','))

header

'id_caso,respuesta'